# Predicing Healthcare related infections
### Capstone 2

Data collection, wrangling, joining, initial checks


In [2]:
#Imports
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import plotly.express as px

#### Significance:

"At any given time, about 1 in 25 inpatients have an infection related to hospital care. These infections lead to tens of thousands of deaths and cost the U.S. health care system billions of dollars each year." - https://health.gov/our-work/health-care-quality/health-care-associated-infections

Additional resources
    - overview of HAI https://www.healthypeople.gov/2020/topics-objectives/topic/healthcare-associated-infections

#### Data summary
**Healthcare Associated Infections (HAIs)** (https://data.cms.gov/provider-data/dataset/77hc-ibv8)
<br> - How often patients get an infections while in the hopsital. This measure is categorized into several different types and means of infections (related to equipment, procedures, or location of infection).  It is also compared to a national benchmark for that type of hospital, and normalized based to some degree based on things like how many beds at the hospital, lab methods used, affilition with a medical school, patient age and some others. Top priority HAIs are central line-associated bloodstream infections (CLABSI) and methicillin-resistant Staphylococcus aureus (MRSA) infections.

**Patient survey (HCAHPS)** (https://data.cms.gov/provider-data/topics/hospitals/hcahps#hcahps-star-ratings) 
<br> - this survey is administered to patients at random (not just medicare patients).  This has 19 questions about the hospital + 10 other demographic and screening questions. (details on questions here: https://data.cms.gov/provider-data/topics/hospitals/hcahps#about-the-hcahps-survey) 

**Star rating (from HCAHPS survey results)** 
<br> - Star rating summarizes the patient survey responses by category, and is rolled into a single 'summary star' rating per facility. (details here: https://data.cms.gov/provider-data/topics/hospitals/hcahps#hcahps-star-ratings)

**Timely and Effective Care** (https://data.cms.gov/provider-data/dataset/yv7e-xc69)
<br> - Includes several measures about specific topics, each topic is given a rating based off what has been shown to be best practice or most important with that procedure.  Data are collected from records of medicare and non-medicare patients. Measures include:  cataract surgery outcome, colonoscopy follow-up, heart attack care, emergency department care, preventive care, pregnancy and delivery care, and cancer care.  Each category has different measures (percentage, number of minutes, etc...) **Most relevant measures here is sepsis - "percentage of patients with severe sepsis or septic shock for which a hospital provides appropriate care".**. (more details about the data: https://data.cms.gov/provider-data/topics/hospitals/timely-effective-care)

**Related data** (https://data.cms.gov/provider-data/dataset/yv7e-xc69)
<br> - In case that wasn't enough links, here's one more. Other datasets included in comparing hospitals that have not been used in this study. 

## Initial imports

In [3]:
#Initial load of files (stored locally, downloaded 2/8/2021)
HAI_raw = pd.read_csv('.\data\Healthcare_Associated_infections_-_Hospital.csv', na_values="Not Available")

display(HAI_raw.shape)
display(HAI_raw.sort_values(by="Facility ID").head(2))

(175608, 16)

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure Name,Measure ID,Compared to National,Score,Footnote,Measure Start Date,Measure End Date,Location
1035,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Central Line Associated Bloodstream Infection ...,HAI_1_CILOWER,No Different than National Benchmark,0.301,NaN,01/01/2019,12/31/2019,POINT (-85.361725 31.214058)
5197,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Catheter Associated Urinary Tract Infections (...,HAI_2_CILOWER,Better than the National Benchmark,0.131,NaN,01/01/2019,12/31/2019,POINT (-85.361725 31.214058)


In [4]:
survey_raw = pd.read_csv('.\data\HCAHPS-Hospital.csv', na_values="Not Available", dtype={12:object, 14:object, 17:object, 19:object},  parse_dates=True, infer_datetime_format=True)
display(survey_raw.shape)
display(survey_raw.sort_values(by="Facility ID").head(2))

(454026, 22)

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,HCAHPS Measure ID,HCAHPS Question,...,Patient Survey Star Rating Footnote,HCAHPS Answer Percent,HCAHPS Answer Percent Footnote,HCAHPS Linear Mean Value,Number of Completed Surveys,Number of Completed Surveys Footnote,Survey Response Rate Percent,Survey Response Rate Percent Footnote,Start Date,End Date
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_A_P,"Patients who reported that their nurses ""Alway...",...,NaN,77,NaN,Not Applicable,507.0,NaN,21.0,NaN,01/01/2019,12/31/2019
67,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_CT_UNDER_D_SD,"Patients who ""Disagree"" or ""Strongly Disagree""...",...,NaN,6,NaN,Not Applicable,507.0,NaN,21.0,NaN,01/01/2019,12/31/2019


In [5]:
care_raw = pd.read_csv('.\data\Timely_and_Effective_Care-Hospital.csv', na_values="Not Available", parse_dates=True, infer_datetime_format=True)
display(care_raw.shape)
display(care_raw.sort_values(by="Facility ID").head(2))

(80665, 16)

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Condition,Measure ID,Measure Name,Score,Sample,Footnote,Start Date,End Date
0,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Emergency Department,EDV,Emergency department volume,high,NaN,NaN,01/01/2019,12/31/2019
16,010001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Sepsis Care,SEV_SEP_6HR,Severe Sepsis 6-Hour Bundle,92,63.0,2,01/01/2019,12/31/2019


We now have the following imported as dataframes

 * HAI_raw
 * survey_raw
 * care_raw

## File QA

In [6]:
profile_HAI = ProfileReport(HAI_raw, title="Infections risk table Pandas Profiling Report")
profile_HAI.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
profile_survey = ProfileReport(survey_raw, title="Survey data Pandas Profiling Report", correlations={"cramers": {"calculate": False}})
profile_survey.to_widgets()

Summarize dataset:   0%|          | 0/36 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\pandas_profiling\model\correlations.py:107: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  (include the error message: '{error}')"""


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
profile_care = ProfileReport(care_raw, title="Timley and effective care table Pandas Profiling Report")
profile_care.to_widgets()

Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Summary from pandas-profiling
**HAI (Hospital associated infections)**
* 4878 unique facility IDs
* Constant start and end date (1/1/2019 - 12/31/2019)
* Score and "compared to national" (key columns for prediction) are missing from >40% and 60% of the rows
* There are 6 types of infections (HAIs) each with 6 metrics, for a total of 6x6=36 unique measures that need moved to columns

**Survey**
* Constant start and end date (1/1/2019 - 12/31/2019)
* 93 distict questions

**Care**
* Start and end date are MOSTLY the same with a few outliers from other quarters.
* Score is missing in 53% of rows
* one of the conditions is "sepsis care", which likley has a high correlation with infections since sepsis is caused by infections.
    

## Wrangle, reshape, and combine files

Files need formatted as 1 row per facility.  Resulting files will be wider instead of longer to prep for joining by Facility ID

### Reshape and clean the HAI data

The main thing this file needs is pivoted.  The Scores and Measures need to be split out, making the dataFrame wide instead of tall.  I'll also drop facilities that have no score data at all.

In [9]:
#First reshape the HAI table
#Step 1. drop measures name, as this can be looked up with the Measure_ID, which is more concise for column names.
#        Save a cross-reference table of Measure_ID and Measure Name for future readibilty and reference
HAI_Names_ID_xref = HAI_raw[['Measure Name', 'Measure ID']].drop_duplicates(keep='first')
HAI_Names_ID_xref.sort_values(by='Measure ID', inplace=True)
HAI_Names_ID_xref.set_index('Measure ID', inplace=True)
print("Shape of HAI_Names_ID_xref:", HAI_Names_ID_xref.shape )

HAI_clean = HAI_raw.drop(['Measure Name', 'Measure End Date'], axis='columns')
print("\n Remaining columns in HAI_clean:\n", HAI_clean.columns)

Shape of HAI_Names_ID_xref: (36, 1)

 Remaining columns in HAI_clean:
 Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Phone Number', 'Measure ID', 'Compared to National',
       'Score', 'Footnote', 'Measure Start Date', 'Location'],
      dtype='object')


In [10]:
#Drop all rows with no score.  These won't be any good to us as score is the primary key indicator
print("Nulls in Score BEFORE drop:", HAI_clean['Score'].isnull().sum())
HAI_clean.dropna(subset=['Score'], how='any', axis='rows',inplace=True)
print("Nulls in Score AFTER drop:", HAI_clean['Score'].isnull().sum())

Nulls in Score BEFORE drop: 76791
Nulls in Score AFTER drop: 0


In [11]:
#Make the HAI data Tidy
#Take the 36 unique measures_IDs and convert to columns showing the score. 
HAI_pivot = HAI_clean.pivot(index="Facility ID", columns="Measure ID", values=["Score"] ) 
HAI_pivot.columns = HAI_pivot.columns.droplevel(0)  #Referenced: https://stackoverflow.com/questions/22233488/pandas-drop-a-level-from-a-multi-level-column-index

#Add a singe columns per measure (6) for 'compared to national average'.
#  Compared to national only applies to measures IDs ending in "SIR" so I'll remove the rest for this step
HAI_ComparedToNational = HAI_clean[HAI_clean['Measure ID'].str.endswith('SIR')]
HAI_ComparedToNational_pivot = HAI_ComparedToNational.pivot(index="Facility ID", columns="Measure ID", values=["Compared to National"] ) 
HAI_ComparedToNational_pivot.columns = HAI_ComparedToNational_pivot.columns.droplevel()

#Join the two pivot tables back by Facility ID
HAI_tidy = pd.merge(HAI_pivot, HAI_ComparedToNational_pivot, on='Facility ID', how="inner", suffixes=('_Score','_ComparedToNational'))
#print("Interm step- should have 42 data + 1 ID columns. Shape (rows, cols):", HAI_tidy.shape)

#Assemble to location data
locationColumns = ['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Phone Number', 'Location']
        
locationData = HAI_clean[locationColumns].drop_duplicates(keep='first')
assert locationData.duplicated().sum() == 0

#Join location data back in after pivots
HAI_tidy = pd.merge(HAI_tidy, locationData, on='Facility ID', how="left")
#This dropped the footnotes column, which is fine, but needs more analysis to see if some key information wasn't included

#Final table should have 36+6 new columns, and only one row per facility. 
print("Now has 42 data columns and 9 descriptor columns. Shape (rows, cols):", HAI_tidy.shape)
print("Column names:\n", HAI_tidy.columns)


Now has 42 data columns and 9 descriptor columns. Shape (rows, cols): (3096, 51)
Column names:
 Index(['Facility ID', 'HAI_1_CILOWER', 'HAI_1_CIUPPER', 'HAI_1_DOPC',
       'HAI_1_ELIGCASES', 'HAI_1_NUMERATOR', 'HAI_1_SIR_Score',
       'HAI_2_CILOWER', 'HAI_2_CIUPPER', 'HAI_2_DOPC', 'HAI_2_ELIGCASES',
       'HAI_2_NUMERATOR', 'HAI_2_SIR_Score', 'HAI_3_CILOWER', 'HAI_3_CIUPPER',
       'HAI_3_DOPC', 'HAI_3_ELIGCASES', 'HAI_3_NUMERATOR', 'HAI_3_SIR_Score',
       'HAI_4_CILOWER', 'HAI_4_CIUPPER', 'HAI_4_DOPC', 'HAI_4_ELIGCASES',
       'HAI_4_NUMERATOR', 'HAI_4_SIR_Score', 'HAI_5_CILOWER', 'HAI_5_CIUPPER',
       'HAI_5_DOPC', 'HAI_5_ELIGCASES', 'HAI_5_NUMERATOR', 'HAI_5_SIR_Score',
       'HAI_6_CILOWER', 'HAI_6_CIUPPER', 'HAI_6_DOPC', 'HAI_6_ELIGCASES',
       'HAI_6_NUMERATOR', 'HAI_6_SIR_Score', 'HAI_1_SIR_ComparedToNational',
       'HAI_2_SIR_ComparedToNational', 'HAI_3_SIR_ComparedToNational',
       'HAI_4_SIR_ComparedToNational', 'HAI_5_SIR_ComparedToNational',
       'HAI_6_S

### Wrangle the survey star ratings file

This file includes 93 unique metrics.  It also includes number of respondants and other items.

Luckily, the CDC has aggregated these into 2 helpful buckets - Star ratings and linear scores.  Many QC steps are performed on these aggregated measures, and I'll be leveraging that work here.  I'll only save the star ratings and linear scores.

Each of the below 10 items have a star rating and a linear score, which should be 20 new columns for the final data set. 
<br>Taken from https://hcahpsonline.org/globalassets/hcahps/star-ratings/tech-notes/hcahps_stars_tech_notes_july_2020.pdf

**HCAHPS Composite Measures (aggregated from multiple questions)**

1. Communication with Nurses (Q1, Q2, Q3)
2. Communication with Doctors (Q5, Q6, Q7)
3. Responsiveness of Hospital Staff (Q4, Q11)
4. Communication about Medicines (Q16, Q17)
5. Discharge Information (Q19, Q20)
6. Care Transition (Q23, Q24, Q25)

**HCAHPS Individual Items**

7. Cleanliness of Hospital Environment (Q8)
8. Quietness of Hospital Environment (Q9)

**HCAHPS Global Items**

9. Hospital Rating (Q21)
10. Recommend the Hospital (Q22)


#### Remove non-star or linear score data

In [54]:
#Regex string reminder referenced: https://stackoverflow.com/questions/26577516/how-to-test-if-a-string-contains-one-of-the-substrings-in-a-list-in-pandas
survey_clean = survey_raw[survey_raw['HCAHPS Measure ID'].str.contains('STAR|LINEAR')]
survey_clean['HCAHPS Measure ID'].unique()  #Should all contain either star scores or linear score

array(['H_COMP_1_LINEAR_SCORE', 'H_COMP_1_STAR_RATING',
       'H_COMP_2_LINEAR_SCORE', 'H_COMP_2_STAR_RATING',
       'H_COMP_3_LINEAR_SCORE', 'H_COMP_3_STAR_RATING',
       'H_COMP_5_LINEAR_SCORE', 'H_COMP_5_STAR_RATING',
       'H_COMP_6_LINEAR_SCORE', 'H_COMP_6_STAR_RATING',
       'H_COMP_7_LINEAR_SCORE', 'H_COMP_7_STAR_RATING',
       'H_CLEAN_LINEAR_SCORE', 'H_CLEAN_STAR_RATING',
       'H_QUIET_LINEAR_SCORE', 'H_QUIET_STAR_RATING',
       'H_HSP_RATING_LINEAR_SCORE', 'H_HSP_RATING_STAR_RATING',
       'H_RECMND_LINEAR_SCORE', 'H_RECMND_STAR_RATING', 'H_STAR_RATING'],
      dtype=object)

#### Pivot Stars and linear score data

In [66]:
#Extract the measures ID and question for future reference
survey_ID_Questoin_Xref = survey_clean[['HCAHPS Measure ID', 'HCAHPS Question']].drop_duplicates()

#The stars and linear scores each have their own column, 
#steps are pivot, remove index, and remove entire columns of "Not Applicable" values
survey_pivot = survey_clean.pivot(index="Facility ID", columns='HCAHPS Measure ID', values=['Patient Survey Star Rating', 'HCAHPS Linear Mean Value'])

survey_pivot.columns = survey_pivot.columns.droplevel()

#Drop columns of "Not Applicable"

#Drop any rows that are 100% null


HCAHPS Measure ID,H_CLEAN_LINEAR_SCORE,H_CLEAN_STAR_RATING,H_COMP_1_LINEAR_SCORE,H_COMP_1_STAR_RATING,H_COMP_2_LINEAR_SCORE,H_COMP_2_STAR_RATING,H_COMP_3_LINEAR_SCORE,H_COMP_3_STAR_RATING,H_COMP_5_LINEAR_SCORE,H_COMP_5_STAR_RATING,...,H_COMP_6_STAR_RATING,H_COMP_7_LINEAR_SCORE,H_COMP_7_STAR_RATING,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_QUIET_LINEAR_SCORE,H_QUIET_STAR_RATING,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING
Facility ID,,,,,,,,,,,,,,,,,,,,,
010001,Not Applicable,3,Not Applicable,3,Not Applicable,3,Not Applicable,3,Not Applicable,3,...,Not Applicable,82,Not Applicable,88,Not Applicable,86,Not Applicable,90,Not Applicable,Not Applicable
010005,Not Applicable,2,Not Applicable,3,Not Applicable,4,Not Applicable,3,Not Applicable,3,...,Not Applicable,81,Not Applicable,88,Not Applicable,85,Not Applicable,88,Not Applicable,Not Applicable
010006,Not Applicable,2,Not Applicable,3,Not Applicable,3,Not Applicable,1,Not Applicable,2,...,Not Applicable,80,Not Applicable,86,Not Applicable,87,Not Applicable,87,Not Applicable,Not Applicable
010007,Not Applicable,3,Not Applicable,3,Not Applicable,3,Not Applicable,3,Not Applicable,4,...,Not Applicable,83,Not Applicable,88,Not Applicable,88,Not Applicable,88,Not Applicable,Not Applicable
010008,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,...,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,Not Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670133,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,...,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,Not Applicable
670134,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,...,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,Not Applicable
670135,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,...,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,NaN,Not Applicable,Not Applicable


#### Join back with tidy HAI file

### Wrangle the Timely and Effective Care file

The only data of interested/easily extractable and likley related here is the Sepsis score.

Below I extract the Sepsis treatment score, discard the rest, and join it to the HAI_tidy dataframe

In [12]:
#Toss out all rows that aren't related to Sepsis
care_clean = care_raw[care_raw.Condition.str.contains("Sepsis")]

#Toss all rows with NaN for Score, these won't help anything
nulls = care_clean.Score.isnull().sum()
care_clean = care_clean.dropna(subset=['Score'], how='any', axis='rows')
print("Removed nulls from sepsis, which was",nulls, "or", round(nulls/(len(care_clean)+nulls),3)*100, "% of the rows")

Removed nulls from sepsis, which was 9802 or 41.3 % of the rows


In [13]:
#Look at footnotes
print("Count of footnote types")
care_clean[['Footnote','Facility ID']].groupby(by='Footnote').count()

Count of footnote types


,Facility ID
Footnote,
2,7222
"2, 3",344
3,311


Footnote meanings:

2. Data submitted were based on a sample of cases/patients.
3. Results are based on a shorter time period than required.

Lots of twos is not an issue, but threes might be.  Since its a small portion of the data, I'm going to drop any rows with a three footnote, indicating they didn't sample for the same time window as other hospitals.

In [14]:
#filter out footnote 3 entries, leave 2s and NaNs
care_clean = care_clean[care_clean.Footnote.str.contains('3') != True]

#Make cross reference table of Condition, Measure ID and Measure Name for future reference before dropping
care_ID_Name_Condition_xref = care_clean[['Condition','Measure ID', 'Measure Name']].drop_duplicates()
display(care_ID_Name_Condition_xref)

,Condition,Measure ID,Measure Name
12,Sepsis Care,SEP_1,Appropriate care for severe sepsis and septic ...
13,Sepsis Care,SEP_SH_3HR,Septic Shock 3-Hour Bundle
15,Sepsis Care,SEV_SEP_3HR,Severe Sepsis 3-Hour Bundle
16,Sepsis Care,SEV_SEP_6HR,Severe Sepsis 6-Hour Bundle
31,Sepsis Care,SEP_SH_6HR,Septic Shock 6-Hour Bundle


In [37]:
#Cross check if we have any dates that are not all of 2019 (removing footnote 3 should have done that for us)
care_clean[["Start Date", "End Date"]].nunique()

Start Date    1
End Date      1
dtype: int64

#### Effect of sample size

In [22]:
#plt1 = care_clean['Sample'].hist(bins=30, range=[0,350])
fig = px.histogram(care_clean, x="Sample", title="Sample size for Sepsis observations")
fig.update_xaxes(range=[0,500])
fig.show()

Not that many hospitals have a small sample count, less than 30 to compute these ranges.  This is something to be aware of going forward, as it may not be representitive, or have a bias on small hospitals.

In [31]:
#This is going to drop location columns, measures Name, Condition, Sample, Footnote and the dates
#  The Condition and Measure name are already saved in a xref table
#  The location is saved in a locaiton table from the HAI wrangling
#  The only real loss is the Sample column.  It's not worth saving, but worth keeping in mind that small sample sizes 
#  may have a bias effect.
care_clean_pivot = care_clean.pivot(index="Facility ID", values='Score', columns='Measure ID')
care_clean_pivot

Measure ID,SEP_1,SEP_SH_3HR,SEP_SH_6HR,SEV_SEP_3HR,SEV_SEP_6HR
Facility ID,,,,,
010001,60,88,NaN,77,92
010005,68,92,84,79,97
010006,33,76,NaN,60,67
010007,98,98,NaN,100,100
010012,88,95,NaN,92,99
...,...,...,...,...,...
670107,25,NaN,NaN,75,23
670108,70,94,NaN,85,86
670120,83,96,86,91,97


In [32]:
#Join it back to the tidy HAI table

#Left join means we won't lose any HAI data, but may have blanks where the Sepsis data didn't exist for a hospital
HAI_tidy = pd.merge(HAI_tidy, care_clean_pivot, on='Facility ID', how="left")

#Final table should have 51 (old) + 5 (new) columns, and only one row per facility. 
print("Now has 47 data columns and 9 descriptor columns. Shape (rows, cols):", HAI_tidy.shape)
print("Column names:\n", HAI_tidy.columns)

Now has 47 data columns and 9 descriptor columns. Shape (rows, cols): (3096, 56)
Column names:
 Index(['Facility ID', 'HAI_1_CILOWER', 'HAI_1_CIUPPER', 'HAI_1_DOPC',
       'HAI_1_ELIGCASES', 'HAI_1_NUMERATOR', 'HAI_1_SIR_Score',
       'HAI_2_CILOWER', 'HAI_2_CIUPPER', 'HAI_2_DOPC', 'HAI_2_ELIGCASES',
       'HAI_2_NUMERATOR', 'HAI_2_SIR_Score', 'HAI_3_CILOWER', 'HAI_3_CIUPPER',
       'HAI_3_DOPC', 'HAI_3_ELIGCASES', 'HAI_3_NUMERATOR', 'HAI_3_SIR_Score',
       'HAI_4_CILOWER', 'HAI_4_CIUPPER', 'HAI_4_DOPC', 'HAI_4_ELIGCASES',
       'HAI_4_NUMERATOR', 'HAI_4_SIR_Score', 'HAI_5_CILOWER', 'HAI_5_CIUPPER',
       'HAI_5_DOPC', 'HAI_5_ELIGCASES', 'HAI_5_NUMERATOR', 'HAI_5_SIR_Score',
       'HAI_6_CILOWER', 'HAI_6_CIUPPER', 'HAI_6_DOPC', 'HAI_6_ELIGCASES',
       'HAI_6_NUMERATOR', 'HAI_6_SIR_Score', 'HAI_1_SIR_ComparedToNational',
       'HAI_2_SIR_ComparedToNational', 'HAI_3_SIR_ComparedToNational',
       'HAI_4_SIR_ComparedToNational', 'HAI_5_SIR_ComparedToNational',
       'HAI_6_S

In [16]:
#check there are no duplicate Facility IDs in final DF (for proper join results)
assert HAI_tidy.duplicated(subset=['Facility ID']).sum() == 0

## Join files on Facility ID

## Summary

HAI

* Initially had 4878 facilities:
* 3096 after removing facilities with no HAI score.

Timley and effective care

* Removed all but the Sepsis data
* Dropped the "Samples" size column, which may point to some facilities with a small sample size bias.  